Import Libraries
TO DO
1) Add larger test dataset
2) Add Speed Tests
3) Create Real Polygons for Testing Purpose

In [1]:
import numpy as np
from osgeo import gdal
from scipy import stats
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor
import matplotlib.pyplot as plt

In [2]:
## Warning You will need 65 gb to create the memory map for the CU imper sum down data type

In [3]:
Grid_Name  = "data\\nlcd_2011_impervious_2011_edition_2014_10_10\\nlcd_2011_impervious_2011_edition_2014_10_10.img"
memory_map_Name = "data\\nlcd_imp.mmp"
Shapefile_Name = "test\\testPolys.shp"

In [21]:
r = gdal.Open(Grid_Name)
r.GetGeoTransform()
r.RasterXSize
band = r.GetRasterBand(1)


1 + float(band.ReadAsArray(10000,20000,1,1))
r.GetRasterBand(1).DataType


data = np.memmap(memory_map_Name, dtype='float32', mode='w+', shape=(r.RasterYSize,r.RasterXSize))
 
 

In [22]:
##Rewrite to do by columns
def dtFZS(ybnd,xbnd,out,band):
    for yw in range(0,ybnd):
        print (yw/ybnd)
        for xw in range(0,xbnd):
            out[yw,xw] = out[yw-1,xw] + float(band.ReadAsArray(xw,yw,1,1))

In [24]:
## rmp function to turn row and column major
## into blocks
## 

def rmp(Yt,Xt,Y,X,y,x):
    DY = (floor((Y-1)/y)) * y; dy = Y - DY;
    DX = (floor((X-1)/x)) * x; dx = X - DX;
 
    p1 = DX * Yt;
    p2 = DY * x;
    p3 = y * (dx-1) ;
    p4 = dy   
    pos = p1 + p2 + p3 + p4;
    return(int(pos-1));


In [127]:
# Function to read the raster as arrays for the chosen block size.
def create_sum_down(x_block_size, y_block_size,raster,mmp):
    ds = gdal.Open(raster)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    blocks = 0
    for y in range(1, ysize, y_block_size):
        print(y/ysize)
        if y + y_block_size < ysize:
            rows = y_block_size
        else:
            rows = ysize - y
        for x in range(0, xsize, x_block_size):
            if x + x_block_size < xsize:
                cols = x_block_size
            else:
                cols = xsize - x
            array = band.ReadAsArray(x, y, cols, rows)
            ##print(cols)
            #print(y)           
            mmp[y:y+rows,x:x+cols] = array  ## + mmp[y-1:y+rows-1,x:x+cols]
            for ywi in range(0,rows):
                for xwi in range(0,cols):
                    mmp[y+ywi,x+xwi] += mmp[y+ywi-1,x+xwi]   
            del array
            blocks += 1
    band = None
    ds = None


In [ ]:
create_sum_down(256,256,Grid_Name,data)

9.576342603232974e-06
0.002461120049030874
0.0049126637554585155
0.007364207461886156
0.009815751168313798
0.012267294874741439
0.01471883858116908
0.017170382287596722
